In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%cd "/content/drive/My Drive/mumbai_hackathon_seer"


/content/drive/My Drive/mumbai_hackathon_seer


In [3]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os

model = VGG16(weights='imagenet', include_top=False)
#model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [21]:
vgg16_feature_list = []
subdir = "images_mens_shirts_segmented/"
#for idx, dirname in enumerate(subdir):
filenames = os.listdir(subdir)

for i, fname in enumerate(filenames):

    #img = image.load_img(img_path, target_size=(224, 224))
    img = image.load_img(subdir+fname, target_size=(224, 224))
    
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature).flatten()
    vgg16_feature_list.append(vgg16_feature_np)
        
vgg16_feature_list_np = np.array(vgg16_feature_list)
kmeans = KMeans(n_clusters=40, random_state=0).fit(vgg16_feature_list_np)
print(kmeans)

from sklearn.externals import joblib
filename = 'kmeans_model.pb'
joblib.dump(kmeans, filename)

predictions = kmeans.predict(vgg16_feature_list_np)
np.save('predictions.npy',predictions)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=40, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)


In [0]:
a = np.load('predictions.npy')
subdir = "images_mens_shirts_segmented/"
filenames = os.listdir(subdir)
d = dict()
for i, a_item in enumerate(a):
  if a_item not in d: 
    d[a_item] = []
  d[a_item].append(filenames[i])
for d_key in d:
  print(str(d_key) + ':' + str(d[d_key]))

In [22]:
loaded_model = joblib.load(filename)
result = loaded_model.predict(vgg16_feature_list_np)
print(result)

[17 21  2 24 24 27  1 32  8 24 36 30 17 11 24 11  2 24 29 29 20 29 14 11
 24 24 11  2  5 14 24  1 37  2 14 22 22 30 30  7 11 27 29  2 38  4  1 15
  6  8 22  7 32 30 27 14  7 24 15 37 24 15 24  2  2 29  1 29 16  1  2 11
 39 14 29  1  7 33  2 24  1 28 29  5 11 30 19 23 10 11  8 14  8  2 30 11
  1  4 13  1 30  8 14  2 17 17  4 17 25  2  1  1  2 35 27  2 21 13  2  2
  2  3 24 14 29 14  3 24 18  1  2 13  8 26  2  1  5 24  2 22  2  2  2  1
 24  8 12  2  2 14 15  2  7  7  7  7 10  7  2  2 13  2  2  2  1  1  2  8
 14 34  2 14  7 37  0  2  2  2 34  2 24  2  2  2 11  2  7 30 34 14 15 37
  2  2  2 31 14  2  9  2  9 19  2  1  2 25  8 20 23 14  2  2 38  7 38]
